In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PC2 : Population dynamics

## The Lotka-Volterra model

By representing by $u_1(t)$ the total mass of plants and $u_2(t)$ the total mass of herbivores at the moment $t$, Lotka introduces the following model :

$$
\left\{\begin{aligned}
\mathrm{d}_t u_1 & = u_1 \, (1-u_2)\\
\mathrm{d}_t u_2 & = u_2 \, (-\alpha+u_1)
\end{aligned}\right.
$$

In [ ]:
class lotka_model:

    def __init__(self, alpha):
        self.alpha = alpha

    def fcn(self, t, u):
        u1, u2 = u
        alpha = self.alpha
        u1_dot = u1 * (1 - u2)
        u2_dot = u2 * (-alpha + u1)
        return (u1_dot, u2_dot)

In [ ]:
alpha = np.arange(0.5,2.6,0.2)

lm = lotka_model(alpha[0])
fcn = lm.fcn

uini = (0.5, 0.5)
tini = 0
tend = 200

tol = 1.e-8

sol = solve_ivp(fcn, (tini, tend), uini, rtol=tol, atol=tol)
u1 = sol.y[0]; u2 = sol.y[1]
inv = u1 + u2 - np.log(u2 * np.power(u1,alpha[0]))

fig = make_subplots(rows=3, cols=1, subplot_titles=("Phase portrait", "Solutions", "Invariant"), vertical_spacing=0.12)
fig.add_trace(go.Scatter(x=u1, y=u2, showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=u1, name="u1"), row=2, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=u2, name="u2"), row=2, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=inv, showlegend=False), row=3, col=1)

#create slider
steps = []
for alpha_i in alpha:
    lm = lotka_model(alpha_i)
    fcn = lm.fcn
    sol = solve_ivp(fcn, (tini, tend), uini, rtol=tol, atol=tol)
    u1 = sol.y[0]; u2 = sol.y[1]
    inv = u1 + u2 - np.log(u2 * np.power(u1,alpha_i))
    step = dict(method="update", label = f"{alpha_i:.2f}", args=[{"x": [u1,sol.t,sol.t,sol.t], 
                                                                  "y": [u2, u1, u2, inv]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'alpha = '}, steps=steps)]

fig.update_xaxes(title_text="u1", range=[0, 8], row=1)
fig.update_yaxes(title_text="u2", range=[0, 5], row=1)
fig.update_xaxes(title_text="t", row=2)
fig.update_yaxes(title_text="t", range=[0, 8], row=2)
fig.update_xaxes(title_text="t", row=3)
fig.update_yaxes(tickformat='.7f', row=3)
fig.update_layout(sliders=sliders, height=1000, legend=dict(x=0.01, y=0.58, bgcolor='rgba(0,0,0,0.1)'))
fig['layout']['sliders'][0]['pad']=dict(t=50)

fig.show()

## Adding intraspecific competition

If we modify the Lotka-Voltera model to take into account the competition between prey, we obtain the following model :

$$
\left\{\begin{aligned}
\mathrm{d}_t u_1 & = u_1 \, (1-\frac{u_1}{K}-u_2)\\
\mathrm{d}_t u_2 & = (u_1 - \alpha) u_2
\end{aligned}\right.
$$

In [ ]:
class lotka_competitive_model:

    def __init__(self, alpha, K):
        self.alpha = alpha
        self.K = K

    def fcn(self, t, u):
        u1, u2 = u
        alpha = self.alpha
        K = self.K
        u1_dot = u1 * (1 - u1/K - u2)
        u2_dot = (u1 - alpha) * u2
        return (u1_dot, u2_dot)

In [ ]:
alpha = np.arange(0.1,3.1,0.1)
K = 2
lcm = lotka_competitive_model(alpha[0], K)
fcn = lcm.fcn

uini = (1., 1.)
tini = 0
tend = 70

tol = 1.e-8

sol = solve_ivp(fcn, (tini, tend), uini, rtol=tol, atol=tol)
u1 = sol.y[0]; u2 = sol.y[1]

fig = make_subplots(rows=2, cols=1, subplot_titles=("Phase portrait", "Solutions"), vertical_spacing=0.14)
fig.add_trace(go.Scatter(x=u1, y=u2, showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=u1, name="u1"), row=2, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=u2, name="u2"), row=2, col=1)

#create slider
steps = []
for alpha_i in alpha:
    lcm = lotka_competitive_model(alpha_i, K)
    fcn = lcm.fcn
    sol = solve_ivp(fcn, (tini, tend), uini, rtol=tol, atol=tol)
    u1 = sol.y[0]; u2 = sol.y[1]
    step = dict(method="update", label = f"{alpha_i:.2f}", args=[{"x": [u1,sol.t,sol.t], "y": [u2, u1, u2]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'alpha = '}, steps=steps)]

fig.update_xaxes(title_text="u1", range=[-0.1, 2.5], row=1)
fig.update_yaxes(title_text="u2", range=[-0.1, 2.5], row=1)
fig.update_xaxes(title_text="t", row=2)
fig.update_yaxes(range=[-0.1, 2.5], row=2)
fig.update_layout(sliders=sliders, height=800, legend=dict(x=0.88, y=0.38, bgcolor='rgba(0,0,0,0.1)'))
fig['layout']['sliders'][0]['pad']=dict(t=50)

fig.show()

## Rosenzweig-MacArthur model

American ecologists Robert Robert MacArthur (1930-1972) et Michael L. Rosenzweig studied the following prey-predator model :

$$
\left\{\begin{aligned}
\mathrm{d}_t u_1  & = u_1 \, \Big( 1-\frac{u_1}{K} \Big) - \frac{u_1 u_2}{1+u_1}\\
\mathrm{d}_t u_2  & = u_2 \Big( \frac{u_1}{1+u_1} - \alpha \Big)  
\end{aligned}\right.
$$

In [ ]:
class rosenzweig_model:

    def __init__(self, alpha, K):
        self.alpha = alpha
        self.K = K

    def fcn(self, t, u):
        u1, u2 = u
        alpha = self.alpha
        K = self.K
        u1_dot = u1 * (1 - (u1/K)) - ((u1*u2)/(1+u1))
        u2_dot = u2 * ((u1/(1+u1)) - alpha)
        return (u1_dot, u2_dot)

In [ ]:
alpha = np.arange(0.2,0.65,0.05)
K = 2
rm = rosenzweig_model(alpha[0], K)
fcn = rm.fcn

uini = (1., 1.)
tini = 0
tend = 200

tol = 1.e-8

sol = solve_ivp(fcn, (tini, tend), uini, rtol=tol, atol=tol)
u1 = sol.y[0]; u2 = sol.y[1]

fig = make_subplots(rows=2, cols=1, subplot_titles=("Phase portrait", "Solutions"), vertical_spacing=0.15)
fig.add_trace(go.Scatter(x=u1, y=u2, showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=u1, name="u1", mode="lines"), row=2, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=u2, name="u2", mode="lines"), row=2, col=1)

#create slider
steps = []
for alpha_i in alpha:
    rm = rosenzweig_model(alpha_i, K)
    fcn = rm.fcn
    sol = solve_ivp(fcn, (tini, tend), uini, rtol=tol, atol=tol)
    u1 = sol.y[0]; u2 = sol.y[1]
    step = dict(method="update", label = f"{alpha_i:.2f}", args=[{"x": [u1,sol.t,sol.t], "y": [u2, u1, u2]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'alpha = '}, steps=steps)]

fig.update_xaxes(title_text="u1", range=[-0.1, 2.5], row=1)
fig.update_yaxes(title_text="u2", range=[-0.1, 2.5], row=1)
fig.update_xaxes(title_text="t", row=2)
fig.update_yaxes(range=[-0.1, 2.5], row=2)
fig.update_layout(sliders=sliders, height=800, legend=dict(x=0.88, y=0.38, bgcolor='rgba(0,0,0,0.1)'))
fig['layout']['sliders'][0]['pad']=dict(t=50)

fig.show()